# Multiperiod Binomial Model

We are given a **derivative security** (either by formula or by explicit values) that takes on different values based on different realizations of a certain stock or asset. 

Our goal is to sell an option and then invest the profit into the **money market** (just gets interest `r`) or the **stock** (changes based on "coin flip" chance), such that our profits match the outcomes of the derivative security exactly. Determining that sell price will guarantee that there is no arbitrage in the market. 

Because this a multi-period situation, we can make a decision at every time step after seeing how the market moved. We can solve this exactly using dynamic programming / recursion.

# The Setup

We set up the example using a stock `s` that evolves over `N` periods. We also have access to a money market with interest rate `r`. For this problem, we assume that `u` = `1/d`, so that the growth and decay rates cancel each other out in each time period. This crucial simplification simplifies state tabulation since it shrinks the total number of stock price possibilties to `2N+1` options. Thus, we can view state as all possible enumerations of stock price `s` and not realizations of the world (sequence of HTHTH...).

In [24]:
N = 3  # number of periods
r = 0.25  # interest rate for money market
s0 = 4  # initial value of the stock
u = 2  # rate of growth of stock on a "head"
d = 1 / 2  # rate of decrease of stock on a "tail"

assert d < 1 + r < u # no arbertrage condition

Next we define the derivative security. A few examples are listed and defined below.

**option**: a contract which gives the owner the right to sell (**put**)/buy(**call**) or a stock a certain price

note: analyzing a lookback_put requires enumerating all realizations of the world and not stock prices since the stock-price paths are relevant for the final calculations. Although effficient approaches exist, I skip it in this analysis. 

In [33]:
def european_put(strike_price, s_n):
    """
    s_n: the price of the stock at expiration

    A contract which gives the RIGHT to the holder to SELL a stock 
    at price strike_price when the time expires.

    returns the payout to the owner of the contract.  
    """
    return max(strike_price - s_n, 0)

def european_call(strike_price, s_n):
    """
    s_n: the price of the stock at expiration

    A contract which gives the RIGHT to the holder to BUY a stock
    at price strike_price when the time expires. 

    returns the payout to the owner of the contract.  
    """
    return max(s_n - strike_price, 0)

def lookback_put(s: list):
    """
    s: ordered list of n stock valuations including the final value at expiration

    A contract which gives the RIGHT to the holder to SELL a stock 
    at the highest valuation of the stock during its lifetime

    returns the payout to the owner of the contract.  
    """
    return max(s) - s[-1]


We use the european put with a strike price of 5

In [34]:
def ds(s):
    return european_put(5, s) 

# Recursive Solution

First we define the risk-neutral probabilties `p_hat` and `q_hat`

In [35]:
p_hat = (1 + r - d) / (u - d)
q_hat = (u - 1 - r) / (u - d)
assert p_hat + q_hat == 1

Next we follow the recursive formulas presented in the book. Note that a DP/backwards-recursion approach or a memoization approach could be used to speed up the recursion. 

In [36]:
def get_portfolio(s0):
    """
    Returns a tuple:
        - value that the derivative security should be sold at
        - how much stock you should hold
    such that taking this action will guarantee you match the ds.   
    """

    def recurse_value(s, t):
        """
        Recursively calculate the value of the portfolio when the stock is at 
        value s at time t. 
        """
        if (t == N):
            return ds(s)

        return 1 / (1 + r) * (p_hat * recurse_value(s * u, t+1) + q_hat * recurse_value(s * d, t+1))
    
    v0 = recurse_value(s0, 0)
    delta0 = (recurse_value(s0 * u, 1) - recurse_value(s0 * d, 1)) / (u * s0 - d * s0)
    return v0, delta0

Calling the function gives us:
1. The value the ds should be sold at
2. How much stock you should HOLD in order to hedge your bet. If you begin with 0 of that stock, a positive value indiciates that you should buy that stock and a negative value indicates that you should short (i.e. borrow those number of shares and sell them -> u owe stock) 

In [38]:
sell_price, stock_hold = get_portfolio(s0)
verb = "buy" if stock_hold > 0 else "short"
print(f"You should sell the derivative security at ${sell_price:.3f}, and {verb} {abs(stock_hold):.3f} amount of the stock")

You should sell the derivative security at $0.864, and short 0.200 amount of the stock


# Verifying Results

Let's work through the math to verify that we got as our answer makes intuitive sense

1. Actions at Time 0:
- We sell the DS for $0.864
- We short .2 of the stock, gaining .2 * 4 = $0.8
- Thus, our total money is `$1.664` and we owe `.2` stock to the person we borrowed from

